# Instrucciones Postgres

La app espera conectarse a una base de datos de postgres usando psycopg2. 

In [2]:
import psycopg2
import pandas as pd

Para evitar tener que usar psql para crear la base de datos, se agrego el siguiente script. Cambiar contraseña postgres si fuese necesario.

In [3]:
# Conexión a la base de datos predeterminada 'postgres'
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="1234", 
    port="5432"
)

conn.autocommit = True  
cursor = conn.cursor()

# Crear la base de datos 'P2G6S1' = (Proyecto 2, Grupo 6, Seccion 1)
try:
    cursor.execute("create database p2g6s1")
    print("Base de datos creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print("La base de datos ya existe.")
except Exception as e:
    print(f"Error al crear la base de datos: {e}")

cursor.close()
conn.close()

La base de datos ya existe.


Se hace la conexion a la base de datos que se conectara con la app. 

In [6]:
# Coneccion a la nueva base de datos
conn = psycopg2.connect(
    host="localhost",
    dbname="p2g6s1", 
    user="postgres",
    password="1234",
    port="5432"
)

# definir una funcion para ejecutar consultas
def ejecutar_consulta(sql_str, select = False):
    try:
        # Crear un cursor para ejecutar las consultas
        cur = conn.cursor()
        # Ejecutar la consulta
        result = cur.execute(sql_str)   
        # Aplicar commit si la consulta es INSERT, UPDATE o DELETE     
        conn.commit()        
        # Obtener los resultados
        if select:
            rows = cur.fetchall()
            df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
            return df
        else:
            return result
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Revertir si hay un error
    finally:
        cur.close()

Crear las tablas en la base de datos 'p2g6s1'

In [54]:
sql_create_table = """
CREATE TABLE IF NOT EXISTS manga(
    title text,
    description text, 
    rating numeric,
    year numeric,
    tags text, 
    cover text, 
    merge text
);
"""

ejecutar_consulta(sql_create_table)

Llenar la tabla con archivo 'data.csv'

In [55]:
# Llenar la tabla desde el archivo CSV
csv_file_path = "data.csv"
try:
    with open(csv_file_path, 'r') as f:
        copy_query = "copy manga from stdin with csv header"
        cur = conn.cursor()
        cur.copy_expert(copy_query, f) 
        conn.commit()
        cur.close()
    print("Datos insertados exitosamente.")
except Exception as e:
    print(f"Error al insertar datos: {e}")
    conn.rollback()

Datos insertados exitosamente.


Probar si los datos fueron insertados.

In [56]:
sql_str = "select merge from manga limit 10;"
ejecutar_consulta(sql_str, select=True)

,merge
0,Salad Days (Tang LiuZang) - Part 2 The second ...
1,The Master of Diabolism As the grandmaster who...
2,JoJo's Bizarre Adventure Part 7: Steel Ball Ru...
3,A Sign of Affection Yuki is a typical college ...
4,Moriarty the Patriot Before he was Sherlock’s ...
5,Classroom of the Elite (Light Novel) At the pr...
6,"Oshi no Ko In the entertainment industry, lyin..."
7,Who Made Me a Princess The beautiful Athanasia...
8,Demon Slayer: Kimetsu no Yaiba The setting is ...
9,"Omniscient Reader (Novel) One day, our MC find..."


Agregar el Indice Invertido GIN

In [57]:
# Se crea una nueva columna donde se guardara el ts_vector indexado
sql_str1 = "alter table manga add column if not exists merge_vector tsvector;"

# Agregar datos a la nueva columna
sql_str2 = "update manga set merge_vector = to_tsvector('english', merge)"

# Agreegar indice invertido GIN
sql_str3 = "create index if not exists manga_merge_index on manga using GIN(merge_vector)"

# Ejecucion de consultas
ejecutar_consulta(sql_str1)
ejecutar_consulta(sql_str2)
ejecutar_consulta(sql_str3)

Probar indice invertido

In [58]:
sql_str = """  
    select merge, ts_rank_cd(merge_vector, query_w) as rank
    from manga, to_tsquery('Tanjirou | Kimetsu | Nezuko' )
    query_w
    where query_w @@ merge_vector
    order by rank desc limit 10;
"""

ejecutar_consulta(sql_str, True)

,merge,rank
0,Demon Slayer: Kimetsu no Yaiba The setting is ...,0.5
1,Kimetsu no Yaiba Koushiki Spin-off: Kimetsu no...,0.3
2,Kimetsu no Yaiba: Tomioka Giyuu Gaiden Side-st...,0.2
3,Kimetsu Gakuen! This entry currently doesn't h...,0.1
4,Demon Slayer: Kimetsu no Yaiba - Shiawase no H...,0.1
5,Demon Slayer: Kimetsu no Yaiba - Signs from th...,0.1
6,Demon Slayer: Kimetsu no Yaiba - Katahane no C...,0.1
7,Demon Slayer: Kimetsu no Yaiba - Tokubetsu Yom...,0.1
8,Kimetsu no Yaiba: Rengoku Kyoujurou Gaiden Sin...,0.1
9,Demon Slayer: Kimetsu no Yaiba - Stories of Wa...,0.1
